In [1]:
import pandas as pd
import pickle
import numpy as np
import re

# Load Education Data

In [2]:
# Load files from pickle files (see load_data.ipynb for code to download datasets)
census_df = pickle.load(open('census_df.pkl', 'rb'))
status_df = pickle.load(open('status_df.pkl', 'rb'))
grad_df = pickle.load(open('grad_df.pkl', 'rb'))

# Initial Feature Selection

The datasets have a number of features that are unecessary to the analysis since they are just additional identifier flags (i.e street_mailing is just the mailing address of the school). To make the data a little easier to work with, I will drop the unecessary columns. 

Removing columns from census DF:

Keeping ncessch to match datasets. Keeping school name and location address to validate matching. Keeping county_code and tract to create tract id.

In [3]:
census_df.columns

Index(['year', 'fips', 'ncessch', 'school_id', 'school_name', 'leaid',
       'street_mailing', 'city_mailing', 'state_mailing', 'zip_mailing',
       'street_location', 'city_location', 'state_location', 'zip_location',
       'county_code', 'congress_district_id', 'gleaid',
       'geocode_accuracy_detailed', 'geocode_accuracy', 'state_fips_geo',
       'county_fips_geo', 'tract', 'puma', 'block_group', 'longitude',
       'latitude', 'geo_longitude', 'geo_latitude', 'geoid_block',
       'census_region', 'census_division', 'csa', 'cbsa', 'cbsa_name',
       'cbsa_type', 'cbsa_city', 'place_name', 'place_fips', 'geoid_place',
       'class_code', 'upper_chamber', 'state_leg_district_upper',
       'upper_chamber_name', 'lower_chamber', 'state_leg_district_lower',
       'lower_chamber_name', 'lower_chamber_type'],
      dtype='object')

In [4]:
census_df = census_df[['ncessch', 'school_name', 'street_location', 'city_location', 'state_location', 'zip_location', 'county_code', 'tract']]

census_df.shape

(101688, 8)

Status df cleanup:

keeping ncessch for matching. Keeping school name and address for validation of matching. Keeping remaining categories for filtering and analysis. 

In [5]:
status_df.columns

Index(['year', 'ncessch', 'school_id', 'school_name', 'leaid', 'lea_name',
       'state_leaid', 'seasch', 'street_mailing', 'city_mailing',
       'state_mailing', 'zip_mailing', 'street_location', 'city_location',
       'state_location', 'zip_location', 'phone', 'fips', 'latitude',
       'longitude', 'csa', 'cbsa', 'urban_centric_locale', 'county_code',
       'school_level', 'school_type', 'school_status', 'lowest_grade_offered',
       'highest_grade_offered', 'bureau_indian_education', 'title_i_status',
       'title_i_eligible', 'title_i_schoolwide', 'charter', 'magnet',
       'shared_time', 'virtual', 'teachers_fte', 'free_lunch',
       'reduced_price_lunch', 'free_or_reduced_price_lunch', 'elem_cedp',
       'high_cedp', 'middle_cedp', 'ungrade_cedp', 'enrollment',
       'state_leg_district_lower', 'state_leg_district_upper', 'ncessch_num',
       'congress_district_id', 'direct_certification', 'lunch_program'],
      dtype='object')

In [6]:
status_df = status_df[['ncessch', 'school_name', 'street_location', 'city_location', 'state_location', 'zip_location', 'urban_centric_locale', 
                       'school_level', 'school_type', 'school_status', 'title_i_status', 'charter', 'magnet', 'shared_time', 'virtual',
                        'lunch_program', 'free_or_reduced_price_lunch']]

status_df.shape

(101688, 17)

Cleaning up graduation rate df

Keeping ncessch for matching. Keeping  schnam (school name) and stnam (state name) for mathing validation. Keeping category and cohort for filtering. Keeping rate as the target.

In [7]:
grad_df.columns

Index(['SCHOOL_YEAR', 'STNAM', 'FIPST', 'LEAID', 'ST_LEAID', 'LEANM',
       'NCESSCH', 'ST_SCHID', 'SCHNAM', 'CATEGORY', 'COHORT', 'RATE',
       'DATE_CUR'],
      dtype='object')

In [8]:
grad_df = grad_df[['NCESSCH', 'SCHNAM', 'STNAM', 'CATEGORY', 'COHORT', 'RATE']]

grad_df.shape

(193753, 6)

# Join Education Data


In [9]:
# Update grad_df columns to better match the format of the other datasets
grad_df.columns = ['ncessch', 'school_name', 'state', 'cohort_cat', 'cohort_num', 'grad_rate']

In [10]:
# converting ncessch to int to match type before joining
census_df['ncessch'] = census_df['ncessch'].astype('int64')
status_df['ncessch'] = status_df['ncessch'].astype('int64')

In [11]:
# Joining the three datasets
temp_df = grad_df.merge(census_df, on='ncessch', how='left')

temp_df = temp_df.merge(status_df, on='ncessch', how='left')

temp_df.head(5)

,ncessch,school_name_x,state,cohort_cat,cohort_num,grad_rate,school_name_y,street_location_x,city_location_x,state_location_x,...,school_level,school_type,school_status,title_i_status,charter,magnet,shared_time,virtual,lunch_program,free_or_reduced_price_lunch
0,10000500871,Albertville High School,ALABAMA,ALL,375,94,Albertville High School,402 E McCord Ave,Albertville,AL,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,901.0
1,10000500871,Albertville High School,ALABAMA,CWD,19,GE80,Albertville High School,402 E McCord Ave,Albertville,AL,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,901.0
2,10000500871,Albertville High School,ALABAMA,ECD,114,90-94,Albertville High School,402 E McCord Ave,Albertville,AL,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,901.0
3,10000500871,Albertville High School,ALABAMA,HOM,7,LT50,Albertville High School,402 E McCord Ave,Albertville,AL,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,901.0
4,10000500871,Albertville High School,ALABAMA,LEP,67,75-79,Albertville High School,402 E McCord Ave,Albertville,AL,...,3.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,2.0,901.0


In [12]:
# Checking to see where matches didn't come through
temp_df.isnull().sum()

ncessch                            0
school_name_x                      0
state                              0
cohort_cat                         0
cohort_num                         0
grad_rate                          0
school_name_y                    357
street_location_x                357
city_location_x                  357
state_location_x                 357
zip_location_x                   357
county_code                      357
tract                            357
school_name                      357
street_location_y                357
city_location_y                  357
state_location_y                 357
zip_location_y                   357
urban_centric_locale             357
school_level                     357
school_type                      357
school_status                    357
title_i_status                  1609
charter                          357
magnet                          1609
shared_time                     1609
virtual                         1609
l

## Review Failed Matches

In [13]:
# Take a look at what schools didn't match

ids = temp_df[temp_df['tract'].isnull()]['ncessch'].unique()

no_match = []

for id in ids:
    no_match.append(grad_df[grad_df['ncessch'] == id]['school_name'].unique()[0])

print(f'There are {len(no_match)} schools missing a match')

There are 49 schools missing a match


In [14]:
# Check to see if I can find a match on name for any of the schools in either of the other datasets
census_data = []
status_data = []
not_found = []
for school in no_match:
    if len(census_df[census_df['school_name'] == school]) > 0:
        census_data.append(school)
    if len(status_df[status_df['school_name'] == school]) > 0:
        status_data.append(school)
    else:
        not_found.append(school)

print('Census:', len(census_data), '\nStatus:', len(status_data), '\nNo Match:', len(not_found))

Census: 29 
Status: 29 
No Match: 20


In [15]:
# double check that the 29 in census and status are the same ones
both_match = [i in status_data for i in census_data]

False in both_match

False

In [16]:
# Using the 29 Schools that match, build a df to compare the ncessch numbers
temp_dict = {}
count = 0
for school in census_data:
    census = [i for i in census_df[census_df['school_name'] == school]['ncessch'].unique()]
    grad = [i for i in grad_df[grad_df['school_name'] == school]['ncessch'].unique()]
    status = [i for i in status_df[status_df['school_name'] == school]['ncessch'].unique()]
    temp_dict[count] = [school, grad, census, status]
    count+= 1

temp_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=['school_name', 'grad_data', 'census_data', 'status_data'])

temp_df

,school_name,grad_data,census_data,status_data
0,Santa Cruz County Career Advancement Charter,[60202814200],[69103614200],[69103614200]
1,Pacific Collegiate Charter,[69103608363],[60254208363],[60254208363]
2,Tehama eLearning Academy,[69104310699],[60254410699],[60254410699]
3,Henryville Jr & Sr High School,[180937001531],[180021201531],[180021201531]
4,Silver Creek High School,[180937001536],"[80537001814, 180021301536]","[80537001814, 180021301536]"
5,William W Borden High School,[180937001540],[180021201540],[180021201540]
6,Kent Education CenterBeltline,[261644000888],[268062000888],[268062000888]
7,Lincoln School,[261644005328],"[171857006295, 173123003268, 180345000475, 201...","[171857006295, 173123003268, 180345000475, 201..."
8,Pine Grove Learning Center,[261644008875],[268062008875],[268062008875]
9,Lincoln Developmental Center,[261644008884],[268062008884],[268062008884]


In [17]:
# Add the 20 with no matching school name
print(len(temp_df))

for school in not_found:
    id = [i for i in grad_df[grad_df['school_name'] == school]['ncessch'].unique()]
    temp_list= [school, id, 'No Match', 'No Match']
    temp_df.loc[len(temp_df)] = temp_list

print(len(temp_df))

# Convert to integers to make things easier
temp_df['grad_data'] = temp_df['grad_data'].apply(lambda x: x[0])

temp_df

29
49


,school_name,grad_data,census_data,status_data
0,Santa Cruz County Career Advancement Charter,60202814200,[69103614200],[69103614200]
1,Pacific Collegiate Charter,69103608363,[60254208363],[60254208363]
2,Tehama eLearning Academy,69104310699,[60254410699],[60254410699]
3,Henryville Jr & Sr High School,180937001531,[180021201531],[180021201531]
4,Silver Creek High School,180937001536,"[80537001814, 180021301536]","[80537001814, 180021301536]"
5,William W Borden High School,180937001540,[180021201540],[180021201540]
6,Kent Education CenterBeltline,261644000888,[268062000888],[268062000888]
7,Lincoln School,261644005328,"[171857006295, 173123003268, 180345000475, 201...","[171857006295, 173123003268, 180345000475, 201..."
8,Pine Grove Learning Center,261644008875,[268062008875],[268062008875]
9,Lincoln Developmental Center,261644008884,[268062008884],[268062008884]


## Identify Correct ncessch id

In order to determine the correct ncessch ID, if one existed, I used the nces.ed.gov site to look up the schools. the ncessch IDs are made up of 3 parts: a state code, a district code, and a school ID. In almost all cases where there was not a match it appears that schools had moved to a new district. So while the state and school ID were the same, the district ID had changed causing the mismatch. For instance Edisto High had (45)(03180)(00862) as its original ID, but after redistricting changed to (45)(03910)(00862). Using this structure I was able to identify the matching items in the census and status datsets. 

In the event that no match could be found using the school name or any of the avaialble ncessch IDs I marked it with 'No Match'

In [18]:
# Dictionary of the correct ncessch IDs
temp_dict = {0: 69103614200, 1: 60254208363, 2: 60254410699, 3: 180021201531, 4: 180021301536, 5: 180021201540, 6: 268062000888, 7: 268062005328, 
            8: 268062008875, 9: 268062008884, 10: 268085007799, 11: 360009105927, 12: 360009505994, 13: 410363001756, 14: 450391001192, 15: 450391000196, 
            16: 450391000862, 17: 450391000880, 18: 450391000850, 19: 450391000873 , 20: 450391000876, 21: 'No Match', 22: 500044300210, 
            23: 500044800207, 24: 500044000127, 25: 500044400022, 26: 500044400256, 27: 'No Match', 28: 500044200539, 29: 'No Match', 30: 'No Match',
            31: 'No Match', 32: 'No Match', 33: 'No Match', 34: 'No Match', 35: 'No Match', 36: 'No Match', 37: 'No Match', 38: 'No Match', 39: 'No Match',
            40: 'No Match', 41: 'No Match', 42: 'No Match', 43: 500044500419, 44: 500043100051, 45: 500043200173, 46: 500043300195, 47: 500043900339, 
            48: 500043709272}

## Update grad_df with new ncessch ids

In [19]:
# Update grad_df with new ncessch IDs
def ncessch_match(row):
    '''Function to be used with apply on grad_Df. Used to identify rows with ncessch IDs that didn't have a match and then update to the correct ncessch ID'''
    old_id = row['ncessch']
    if temp_df['grad_data'].isin([old_id]).any():
        ind = temp_df[temp_df['grad_data'] == old_id].index[0]
        new_id = temp_dict[ind]
        return new_id
    else: return old_id

grad_df['ncessch'] = grad_df.apply(ncessch_match, axis=1)

In [20]:
# Spot checking a few schools to make sure the ncessch ID updated
print(grad_df[grad_df['school_name'] == 'Windsor School'])
print(grad_df[grad_df['school_name'] == 'Malan\'s Peak Secondary'])
print(grad_df[grad_df['school_name'] == 'Missisquoi Valley UHSD #7'])

             ncessch     school_name    state cohort_cat  cohort_num grad_rate
177228  500044200539  Windsor School  VERMONT        ALL          59     80-89
177229  500044200539  Windsor School  VERMONT        CWD           8      GE50
177230  500044200539  Windsor School  VERMONT        ECD          22     60-79
177231  500044200539  Windsor School  VERMONT        HOM           3        PS
177232  500044200539  Windsor School  VERMONT        MTR           1        PS
177233  500044200539  Windsor School  VERMONT        MWH          57     70-79
         ncessch             school_name state cohort_cat  cohort_num  \
176360  No Match  Malan's Peak Secondary  UTAH        ALL          21   
176361  No Match  Malan's Peak Secondary  UTAH        CWD           6   
176362  No Match  Malan's Peak Secondary  UTAH        ECD           5   
176363  No Match  Malan's Peak Secondary  UTAH        MBL           5   
176364  No Match  Malan's Peak Secondary  UTAH        MHI           7   
176365  N

## Remove schools with no match found

In [21]:
# Dropping all schools that I could not find a match for
print('Number of rows with no match:', len(grad_df[grad_df['ncessch'] == 'No Match']))
print('Length of Dataframe before dropping:', len(grad_df))

grad_df = grad_df[grad_df['ncessch'] != 'No Match']

print('Length of DataFrame after filetering:', len(grad_df))

Number of rows with no match: 105
Length of Dataframe before dropping: 193753
Length of DataFrame after filetering: 193648


## Second attempt to join datasets

In [22]:
# Convert ncessch back to an integer
grad_df['ncessch'] = grad_df['ncessch'].astype('int64')

In [23]:
edu_df = grad_df.merge(census_df, on='ncessch', how='left')

edu_df = edu_df.merge(status_df, on='ncessch', how='left')

edu_df.head(5)

,ncessch,school_name_x,state,cohort_cat,cohort_num,grad_rate,school_name_y,street_location_x,city_location_x,state_location_x,...,school_level,school_type,school_status,title_i_status,charter,magnet,shared_time,virtual,lunch_program,free_or_reduced_price_lunch
0,10000500871,Albertville High School,ALABAMA,ALL,375,94,Albertville High School,402 E McCord Ave,Albertville,AL,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
1,10000500871,Albertville High School,ALABAMA,CWD,19,GE80,Albertville High School,402 E McCord Ave,Albertville,AL,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
2,10000500871,Albertville High School,ALABAMA,ECD,114,90-94,Albertville High School,402 E McCord Ave,Albertville,AL,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
3,10000500871,Albertville High School,ALABAMA,HOM,7,LT50,Albertville High School,402 E McCord Ave,Albertville,AL,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
4,10000500871,Albertville High School,ALABAMA,LEP,67,75-79,Albertville High School,402 E McCord Ave,Albertville,AL,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0


In [24]:
# Check if any schools still don't have a match

edu_df.isnull().sum()

ncessch                            0
school_name_x                      0
state                              0
cohort_cat                         0
cohort_num                         0
grad_rate                          0
school_name_y                      0
street_location_x                  0
city_location_x                    0
state_location_x                   0
zip_location_x                     0
county_code                        0
tract                              0
school_name                        0
street_location_y                  0
city_location_y                    0
state_location_y                   0
zip_location_y                     0
urban_centric_locale               0
school_level                       0
school_type                        0
school_status                      0
title_i_status                  1252
charter                            0
magnet                          1252
shared_time                     1252
virtual                         1252
l

# Filter Education Data

In [25]:
edu_df.columns

Index(['ncessch', 'school_name_x', 'state', 'cohort_cat', 'cohort_num',
       'grad_rate', 'school_name_y', 'street_location_x', 'city_location_x',
       'state_location_x', 'zip_location_x', 'county_code', 'tract',
       'school_name', 'street_location_y', 'city_location_y',
       'state_location_y', 'zip_location_y', 'urban_centric_locale',
       'school_level', 'school_type', 'school_status', 'title_i_status',
       'charter', 'magnet', 'shared_time', 'virtual', 'lunch_program',
       'free_or_reduced_price_lunch'],
      dtype='object')

In [26]:
# Drop duplicated columns

edu_df = edu_df.drop(columns=['state', 'school_name_y', 'school_name', 'street_location_y', 
                            'city_location_y', 'state_location_y','zip_location_y'])

In [27]:
# Drop suffix from columns

edu_df.rename(columns= {'school_name_x' : 'school_name', 'street_location_x': 'street',  'state_location_x': 'state', 
                        'city_location_x': 'city', 'zip_location_x': 'zip'}, inplace= True)

In [28]:
edu_df.head()

,ncessch,school_name,cohort_cat,cohort_num,grad_rate,street,city,state,zip,county_code,...,school_level,school_type,school_status,title_i_status,charter,magnet,shared_time,virtual,lunch_program,free_or_reduced_price_lunch
0,10000500871,Albertville High School,ALL,375,94,402 E McCord Ave,Albertville,AL,35950,1095,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
1,10000500871,Albertville High School,CWD,19,GE80,402 E McCord Ave,Albertville,AL,35950,1095,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
2,10000500871,Albertville High School,ECD,114,90-94,402 E McCord Ave,Albertville,AL,35950,1095,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
3,10000500871,Albertville High School,HOM,7,LT50,402 E McCord Ave,Albertville,AL,35950,1095,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0
4,10000500871,Albertville High School,LEP,67,75-79,402 E McCord Ave,Albertville,AL,35950,1095,...,3,1,1,3.0,0,0.0,0.0,0.0,2.0,901.0


## Remove closed schools

Now that I have sucessfully joined the dataframes, I can filter out some of the schools that will not aid in the analysis. 

As a first step, I can use school_status to remove any school that was closed or inactive at the beginning of the school year since these schools will either have no graduation data or the graduation data will be for students who moved schools partway through the year which could skew the results. 

One benefit of this is that I have now removed all of the null values for all but one column

In [29]:
# Remove any schools that were closed or inactive at the beginning of the 2018/2019 school year. 

edu_df = edu_df[~(edu_df['school_status'].isin([2, 6]))]

In [30]:
edu_df.shape

(192396, 22)

In [31]:
# Verify that this has removed all of the null values for everything except free_or_reduced_price_lunch
edu_df.isnull().sum()

ncessch                            0
school_name                        0
cohort_cat                         0
cohort_num                         0
grad_rate                          0
street                             0
city                               0
state                              0
zip                                0
county_code                        0
tract                              0
urban_centric_locale               0
school_level                       0
school_type                        0
school_status                      0
title_i_status                     0
charter                            0
magnet                             0
shared_time                        0
virtual                            0
lunch_program                      0
free_or_reduced_price_lunch    14052
dtype: int64

## Remove schools with high probability of students traveling from ther areas

Next, I can filter out schools that are likely to have student populations that may not live in the area that they attend school. These are populations attending specialty schools that would likely involve traveling from another area or remote schools

School Type: Exclude special education, vocational, and other/alternative schools as well as reportable programs since these programs would have a number of factors that would impact graduation rates that 'Regular Schools' would not. (Only include schools with 1 or < 0)

Exclude: Magnet Schools, Shared Time, and Virtual Schools since the student population of these schools may not live in the area their school is located and therefore may have different food access. 

In [32]:
# Drop schools with unwanted types
edu_df = edu_df[(edu_df['magnet'] != 1.0) & (edu_df['shared_time'] != 1.0) & (edu_df['virtual'] != 1.0) & (edu_df['virtual'] != 2) & ~(edu_df['school_type'].isin([2, 3, 4, 5]))]

A note that magent, shared time, and virtual still have values that are missing/not reported (-1) or not applicable (-2). I'll come back to these later.

In [33]:
# Check new values in each column
cols = ['school_type', 'school_status', 'magnet', 'shared_time', 'virtual']

for col in cols:
    print(col, ':', edu_df[col].unique())

school_type : [1]
school_status : [1 5]
magnet : [ 0. -1. -2.]
shared_time : [ 0. -1.]
virtual : [ 0.  3. -1.]


In [34]:
# Check how many unique schools are left after filtering
edu_df['ncessch'].nunique()

17257

In [35]:
len(edu_df)

149281

I can drop school type and school status since they will no longer be needed. 

In [36]:
edu_df.drop(columns=['school_type', 'school_status'], inplace=True)

# Graduation Rate Feature Engineering

In [37]:
# Take a look at remaining features
edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149281 entries, 0 to 193647
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ncessch                      149281 non-null  int64  
 1   school_name                  149281 non-null  object 
 2   cohort_cat                   149281 non-null  object 
 3   cohort_num                   149281 non-null  int64  
 4   grad_rate                    149281 non-null  object 
 5   street                       149281 non-null  object 
 6   city                         149281 non-null  object 
 7   state                        149281 non-null  object 
 8   zip                          149281 non-null  object 
 9   county_code                  149281 non-null  int64  
 10  tract                        149281 non-null  float64
 11  urban_centric_locale         149281 non-null  int64  
 12  school_level                 149281 non-null  int64  
 13 

## Clean Up Grad Rate

There are 4 types of notation for Grad Rate that need to be addressed:

1) A single number: This means that the cohort was large enough that the true graduation rate was reported, we cna just use this number.
2) Two numbers with a dash: This means that the graduation rate was given as a range to protect privacy. For these, we'll take the average of the two numbers as teh graduation rate
3) GE/LE or GT/LT: This means greater/less than or equal to the number provided. For these, since we have no way of getting more accurate numbers, we'll just take the number provided as the rate
4) PS: This means the cohort was small enough that the graduation rate was surpressed. We'll need to determine whether these can be dropped or if we need to impute these

In [38]:
# Look at the values of grad rate

edu_df['grad_rate'].unique()


array(['94', 'GE80', '90-94', 'LT50', '75-79', 'GE50', 'PS', 'GE95',
       'GE90', '85-89', '80-89', '93', '50-59', '70-79', '97', '80-84',
       '40-59', '98', '70-74', '91', '60-79', '60-64', '65-69', '95',
       '96', '90', '88', '92', '60-69', '87', '84', '78', '21-39', '80',
       'LE20', '89', '85', '86', '40-49', '81', '82', '83', '55-59',
       '45-49', '74', '30-34', '20-29', '40-44', '35-39', '61', '77',
       '79', '76', '30-39', 'GE99', '75', '73', '11-19', '68', '57',
       '50-54', '71', '72', '25-29', '20-24', '20', '10-14', '18',
       '15-19', 'LE5', 'LE10', '2', 'LE1', '14', '6-9', '13', '11', '19',
       '17', '31', '29', '10', '9', '16', '22', '21', '41', '37', '58',
       '24', '23', '5', '8', '4', '3', '28', '12', '15', '70', '69', '66',
       '59', '62', '56', '26', '65', '54', '64', '63', '67', '53', '35',
       '39', '60', '49', '52', '40', '48', '25', '33', '55', '43', '27',
       '46', '45', '44'], dtype=object)

In [39]:
# Pull out a graduation rate for all observations
def grad_rate(rate):
    '''Function to assess the string notation for grad rate and convert to a single number
    or null value if data is supressed'''
    if 'PS' in rate:
        return np.nan
    if 'GE' in rate or 'LE' in rate:
        new_rate = re.sub(r'\S["E"]', '', rate)
        return new_rate
    if 'GT' in rate:
        new_rate = re.sub(r'\S["T"]', '', rate)
        return int(new_rate) + 1
    if 'LT' in rate:
        new_rate = re.sub(r'\S["T"]', '', rate)
        return int(new_rate) - 1
    if '-' in rate:
        nums = rate.split('-')
        new_rate = (int(nums[0]) + int(nums[1]))/2
        return new_rate
    else:
        return rate
    
edu_df['grad_num'] = edu_df['grad_rate'].apply(grad_rate)

In [40]:
# Confirm that all values are now either a single number or NaN

edu_df['grad_num'].unique()

array(['94', '80', 92.0, 49, 77.0, '50', nan, '95', '90', 87.0, 84.5,
       '93', 54.5, 74.5, '97', 82.0, 49.5, '98', 72.0, '91', 69.5, 62.0,
       67.0, '96', '88', '92', 64.5, '87', '84', '78', 30.0, '20', '89',
       '85', '86', 44.5, '81', '82', '83', 57.0, 47.0, '74', 32.0, 24.5,
       42.0, 37.0, '61', '77', '79', '76', 34.5, '99', '75', '73', 15.0,
       '68', '57', 52.0, '71', '72', 27.0, 22.0, 12.0, '18', 17.0, '5',
       '10', '2', '1', '14', 7.5, '13', '11', '19', '17', '31', '29', '9',
       '16', '22', '21', '41', '37', '58', '24', '23', '8', '4', '3',
       '28', '12', '15', '70', '69', '66', '59', '62', '56', '26', '65',
       '54', '64', '63', '67', '53', '35', '39', '60', '49', '52', '40',
       '48', '25', '33', '55', '43', '27', '46', '45', '44'], dtype=object)

In [41]:
# Check how many observations are missing values
edu_df['grad_num'].isnull().sum()

50899

# Impute Missing Graduation Rates

There are enough missing values for grad rates that I will want to impute a value rather than drop them. In order to create some sort of similarity, rather than just use the average graduation rate, I will use the average graduation rates for students in the same state, urban centric locale, and sub_cohort since these populations are most likely to be similar in other respects. 

## Calcualte values to impute

First I'll need to get urban_centric_locale in a usable state. I'll do this by grouping the provided numbers into the top level categories

Source:  https://nces.ed.gov/surveys/annualreports/topical-studies/locale/definitions

In [42]:
# First I'll simplify urban centric locale

# Create urban centric locale category
def locale (x):
    if x['urban_centric_locale'] in [1, 2, 3, 4, 11, 12, 13]:
        return 'Urban'
    if x['urban_centric_locale'] in [21, 22, 23]:
        return 'Suburban'
    if x['urban_centric_locale'] in [5, 6, 31, 32, 33]:
        return 'Town'
    if x['urban_centric_locale'] in [7, 8, 41, 42, 43]:
        return 'Rural'
    # Returning a -1 will catch any values that don't match the definitions
    else:
        return -1
    
edu_df['urban_locale_cat'] = edu_df.apply(locale, axis='columns')

In [43]:
# Double check that all values updated
edu_df['urban_locale_cat'].unique()

array(['Town', 'Rural', 'Urban', 'Suburban'], dtype=object)

In [44]:
# Create a pivot table of graduation rates by state, urban locale, and cohort
temp_df = edu_df[edu_df['grad_num'].isnull() == False]

temp_df['grad_num'] = temp_df['grad_num'].astype('int32')

avg_grad = temp_df.pivot_table(index=['state', 'urban_locale_cat', 'cohort_cat'], values='grad_num', aggfunc=lambda x: round(np.mean(x), 2))

avg_grad

/tmp/ipykernel_424/509830829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['grad_num'] = temp_df['grad_num'].astype('int32')


grad_num
state urban_locale_cat cohort_cat          
AK    Rural            ALL            58.97
                       CWD            47.33
                       ECD            55.42
                       HOM            49.50
                       LEP            49.79
...                                     ...
WY    Urban            MAS            50.00
                       MBL            50.00
                       MHI            68.86
                       MTR            50.00
                       MWH            73.29

[2101 rows x 1 columns]

## Fill in missing graduation rates

In [45]:
def grad_rate_fill_na (row):
    if pd.isnull(row['grad_num']) == True:
        state = row['state']
        locale = row['urban_locale_cat']
        cohort = row['cohort_cat']

        if cohort in avg_grad.loc[state].loc[locale].index:
            return avg_grad.loc[state].loc[locale].loc[cohort]['grad_num']
        # This occurs if there are no cohorts of that type anywhere in that state/locale combination who had a graduation rate reported
        else:
            return -1
    else:
        return row['grad_num']
        

edu_df['grad_num_2'] = edu_df.apply(grad_rate_fill_na, axis=1)

## Remove remaining null values

It looks like I managed to impute a value for over 47,000 of the observations and I'm left with only 3,150 observations with no graduation rate. These occured because that particular group is so small and/or spread out that there is no instance within that state/locale combination of that particular cohort category that was large enough to have their graduation rates reported.

On the plus side, there are no schools that don't at least have a graduation rate reported for their total student body. 

At this point, any values I impute for the reaminin 3k observations would be random guesses, and these groups represent only a tiny fraction of the student body, so it is probably not going to add much to the analysis. So at this point, I will drop anything that I was unable to impute a value for. 

In [46]:
print('Prior to imputing values, the number of observations missing a graduation rate was:', len(edu_df[edu_df['grad_num'].isnull()]))

print('After imputing values, the number of observations missing a graduation rate was: ', len(edu_df[edu_df['grad_num_2'] == -1]))

print('The total number of schools with no graduation rates availalbe for the total cohort was: ', len(edu_df[(edu_df['grad_num_2'] == -1) & (edu_df['cohort_cat'] == 'ALL')]))

Prior to imputing values, the number of observations missing a graduation rate was: 50899
After imputing values, the number of observations missing a graduation rate was:  3150
The total number of schools with no graduation rates availalbe for the total cohort was:  0


In [47]:
edu_df = edu_df[edu_df['grad_num_2'] != -1]

In [48]:
len(edu_df)

146131

## Remove Free/Reduced Lunch Feature

All of the remaining null values are in the free_or_reduced_price_lunch feature. While I thought this feature could be interesting, with so many missing values it won't be particularly helpful so I'll just remove it. 

In [49]:
edu_df.drop(columns=['free_or_reduced_price_lunch'], inplace=True)

# Clean and Save Final DataFrame

To make this DataFrame a litte easier to read I'll make a few changes including renaming the cohort categories, drop the columns I'm finished using, and renaming the graduation rate feature. Finally I'll save this as a pickle file so that I can easily load it again later. 

In [50]:
# Create dictionary of new cohort category names
cohort_dict = {'ALL': 'total', 'MAM': 'am_indian/ak_native', 'MAS': 'api', 'MHI': 'hispanic', 'MBL': 'black', 'MWH': 'white', 'MTR': 'two_or_more', "CWD": 'disability',
               'ECD': 'econ_disadvantaged', 'LEP': 'limited_english', 'HOM': 'homeless', 'FCS': 'foster_care'}

#Update the category column
edu_df['cohort_cat'] = edu_df['cohort_cat'].apply(lambda cat: cohort_dict[cat])

edu_df

,ncessch,school_name,cohort_cat,cohort_num,grad_rate,street,city,state,zip,county_code,...,school_level,title_i_status,charter,magnet,shared_time,virtual,lunch_program,grad_num,urban_locale_cat,grad_num_2
0,10000500871,Albertville High School,total,375,94,402 E McCord Ave,Albertville,AL,35950,1095,...,3,3.0,0,0.0,0.0,0.0,2.0,94,Town,94
1,10000500871,Albertville High School,disability,19,GE80,402 E McCord Ave,Albertville,AL,35950,1095,...,3,3.0,0,0.0,0.0,0.0,2.0,80,Town,80
2,10000500871,Albertville High School,econ_disadvantaged,114,90-94,402 E McCord Ave,Albertville,AL,35950,1095,...,3,3.0,0,0.0,0.0,0.0,2.0,92.0,Town,92.0
3,10000500871,Albertville High School,homeless,7,LT50,402 E McCord Ave,Albertville,AL,35950,1095,...,3,3.0,0,0.0,0.0,0.0,2.0,49,Town,49
4,10000500871,Albertville High School,limited_english,67,75-79,402 E McCord Ave,Albertville,AL,35950,1095,...,3,3.0,0,0.0,0.0,0.0,2.0,77.0,Town,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193643,720003002085,JUAN QUIRINDONGO MORELL (SUPERIOR),total,182,80-84,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,...,3,5.0,0,0.0,0.0,0.0,1.0,82.0,Suburban,82.0
193644,720003002085,JUAN QUIRINDONGO MORELL (SUPERIOR),disability,36,60-69,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,...,3,5.0,0,0.0,0.0,0.0,1.0,64.5,Suburban,64.5
193645,720003002085,JUAN QUIRINDONGO MORELL (SUPERIOR),econ_disadvantaged,146,80-84,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,...,3,5.0,0,0.0,0.0,0.0,1.0,82.0,Suburban,82.0
193646,720003002085,JUAN QUIRINDONGO MORELL (SUPERIOR),homeless,1,PS,CALLE Q FERNAL URB EL ROSARIO,VEGA BAJA,PR,00693,72145,...,3,5.0,0,0.0,0.0,0.0,1.0,NaN,Suburban,49.9


In [51]:
# Drop columns no longer needed. 
edu_df.drop(columns=['grad_rate', 'urban_centric_locale', 'grad_num'], inplace=True)

In [52]:
# Rename the new graduation rate feature
edu_df.rename(columns={'grad_num_2': 'grad_rate'}, inplace=True)

In [53]:
# Do one last check of data
edu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146131 entries, 0 to 193647
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ncessch           146131 non-null  int64  
 1   school_name       146131 non-null  object 
 2   cohort_cat        146131 non-null  object 
 3   cohort_num        146131 non-null  int64  
 4   street            146131 non-null  object 
 5   city              146131 non-null  object 
 6   state             146131 non-null  object 
 7   zip               146131 non-null  object 
 8   county_code       146131 non-null  int64  
 9   tract             146131 non-null  float64
 10  school_level      146131 non-null  int64  
 11  title_i_status    146131 non-null  float64
 12  charter           146131 non-null  int64  
 13  magnet            146131 non-null  float64
 14  shared_time       146131 non-null  float64
 15  virtual           146131 non-null  float64
 16  lunch_program     14

In [54]:
edu_df.sample(10)

,ncessch,school_name,cohort_cat,cohort_num,street,city,state,zip,county_code,tract,school_level,title_i_status,charter,magnet,shared_time,virtual,lunch_program,urban_locale_cat,grad_rate
67206,200438000744,Caldwell Secondary School,econ_disadvantaged,8,31 North Osage Street,Caldwell,KS,67022,20191,962300.0,3,1.0,0,0.0,-1.0,0.0,1.0,Rural,50
135100,390451702111,Zanesville High School,two_or_more,32,1701 Blue Ave,Zanesville,OH,43701,39119,911700.0,3,5.0,0,-1.0,0.0,0.0,2.0,Town,84.5
177437,510045000179,VIRGINIA HIGH,api,2,1200 Long Crescent Dr,Bristol,VA,24201,51520,20200.0,3,6.0,0,0.0,0.0,3.0,1.0,Suburban,74.82
5015,40010601892,AAEC - Paradise Valley,econ_disadvantaged,13,3775 E UNION HILLS DR,PHOENIX,AZ,85050,4013,619600.0,3,2.0,1,-1.0,0.0,0.0,0.0,Urban,50
42928,90279001216,Sound School,api,1,60 South Water St.,New Haven,CT,06519,9009,140400.0,3,6.0,0,0.0,0.0,-1.0,2.0,Urban,66.0
140745,401521000713,HUGO HS,total,80,704 North 2nd Street,Hugo,OK,74743,40023,967100.0,3,5.0,0,-2.0,0.0,0.0,2.0,Town,87.0
7724,40340002965,Campo Verde High School,hispanic,71,3870 S QUARTZ ST,GILBERT,AZ,85297,4013,815400.0,3,1.0,0,-1.0,0.0,0.0,1.0,Rural,92.0
148169,421074002043,Girard HS,homeless,3,1135 Lake St,Girard,PA,16417,42049,10201.0,3,4.0,0,0.0,0.0,0.0,1.0,Suburban,49.89
12757,60002200395,Mendota High,total,237,1200 Belmont Ave.,Mendota,CA,93640,6019,8302.0,3,5.0,0,0.0,-1.0,3.0,2.0,Rural,78
174996,484668005323,J M HANKS H S,hispanic,376,2001 LEE TREVINO DR,EL PASO,TX,79935,48141,10307.0,3,5.0,0,0.0,-1.0,-1.0,1.0,Urban,88


In [55]:
# Save final file as a pickle file

edu_df.to_pickle(open('edu_df.pkl', 'wb'))